## Set the path of where the csv files are located, as well as initial import statements

In [1]:
csv_data_path = "/home/quinton/Documents/Projects/COMET_Prestudy/scripts/collection/formal_scripts/output_data_RIPE_01_2017.csv" # **PLACE CSV FILEPATH HERE**
sep = "\t" # csv delimiter used
small_model = True # use smaller ML model for better performance, less accuracy

In [2]:
from pathlib import Path 
import pandas as pd
import numpy as np

# Gensim imports
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy
import spacy
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# visualization
import pyLDAvis
import pyLDAvis.gensim_models

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/quinton/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/home/quinton/Documents/Projects/COMET_Prestudy/scripts/analysis/.venv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


## Functions to handle data loading and data writing

In [3]:
def load_data(filename) -> pd.DataFrame:
    filename = Path(filename)
    if not filename.is_file():
        raise FileNotFoundError

    with open(filename.resolve(), 'r', encoding='utf-8') as file:
        data = pd.read_csv(file, sep=sep)
    return data

def load_data_by_year(directory: Path) -> pd.DataFrame:
    for data_file in Path.glob(Path.joinpath(directory, '*.csv')):
            yield load_data(data_file.resolve())

def write_data(filename: Path, data: pd.DataFrame):
    with open(filename.as_uri(), 'w', encoding='utf-8') as f:
        data.to_csv(f, sep=sep)


In [4]:
# obtain the stop words
stopwords = stopwords.words("english")
stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [5]:
data = load_data(csv_data_path)['content']
data.head()

0    Hi,just a question:any interest to have a prob...
1    Hi Meikel,Sorry for the silence. I had noticed...
2    Hi everyone,accoring to this site, a CentOS 7 ...
3    Hi,I may be wrong but it looks like there are ...
4    Hello everyone, I would like to host a new Atl...
Name: content, dtype: object

# Content Preprocessing

In [6]:
def lemmatize(posts: pd.DataFrame, allowed_post_tags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm" if small_model else "en_core_web_trf", disable=["parser", 'ner'])
    texts_out = []

    for post in posts:
        doc = nlp(post)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_post_tags:
                new_text.append(token.lemma_)
        final = ' '.join(new_text)
        texts_out.append(final)
    return texts_out

lemmatized_data = lemmatize(data.astype(str))
lemmatized_data[0][:10]

'just quest'

In [7]:
def tokenize_lemma(posts: list):
    final = []
    for post in posts:
        new = gensim.utils.simple_preprocess(post, deacc=True)
        final.append(new)
    return final

tokenized_data = tokenize_lemma(lemmatized_data)
tokenized_data[0][:10]

['just',
 'question',
 'interest',
 'probe',
 'sailboat',
 'officially',
 'supported',
 'just',
 'get',
 'probe']

# Generate id2word dictionary

In [8]:
id2word = corpora.Dictionary(tokenized_data)

corpus = []

for text in tokenized_data:
    new = id2word.doc2bow(text)
    corpus.append(new)

corpus[0][:10]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 2),
 (7, 1),
 (8, 2),
 (9, 1)]

# LDA Visualization

In [13]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=15,
random_state=100, update_every=1, chunksize=100, passes=10, alpha="auto")

In [14]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds='mmds', R=30)
vis

/home/quinton/Documents/Projects/COMET_Prestudy/scripts/analysis/.venv/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.355428  0.222715       1        1  43.737126
6     -0.408431 -0.091300       2        1  28.962446
2     -0.192371 -0.080229       3        1   6.211814
9     -0.107544 -0.262572       4        1   6.104313
11     0.312497 -0.165634       5        1   4.442073
0      0.097675 -0.315668       6        1   3.833027
5      0.143705  0.159360       7        1   2.153169
13     0.022323 -0.037302       8        1   1.606707
7     -0.060748  0.370569       9        1   1.322031
14     0.051307  0.054811      10        1   0.447131
8      0.083397  0.042599      11        1   0.349839
12     0.202207  0.042108      12        1   0.296995
3      0.081187  0.011503      13        1   0.283502
10     0.067291  0.026545      14        1   0.155646
4      0.062933  0.022493      15        1   0.094181, topic_info=                Term        Freq       Total Category  logprob  loglift
55       measurement  475.000000  475.000000  Default  30.0000  30.0000
76              ripe  546.000000  546.000000  Default  29.0000  29.0000
15             probe  860.000000  860.000000  Default  28.0000  28.0000
41               dot  168.000000  168.000000  Default  27.0000  27.0000
551          message  128.000000  128.000000  Default  26.0000  26.0000
...              ...         ...         ...      ...      ...      ...
2556       important    0.005426    0.747183  Topic15  -8.3033   2.0426
1898            less    0.005425    0.750104  Topic15  -8.3034   2.0386
2289          option    0.005435    0.781897  Topic15  -8.3015   1.9989
2073            with    0.005422    0.745505  Topic15  -8.3040   2.0440
2426  dnsmeasurement    0.005422    0.744394  Topic15  -8.3040   2.0455

[638 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
561       8  0.468976    abandon
1115      4  0.944840      about
430       4  0.926011      above
466       1  0.724742    account
466       3  0.271778    account
...     ...       ...        ...
118       1  0.237146       year
118       2  0.284575       year
118       4  0.474291       year
174       6  0.989722  yesterday
1500      5  0.840715        yet

[538 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 7, 3, 10, 12, 1, 6, 14, 8, 15, 9, 13, 4, 11, 5])